In [1]:
import requests 

from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from pprint import pprint
from time import sleep
import pandas as pd
import numpy as np

In [2]:
def extrae_datos_meneos (lista_meneos):
    datos = []
    continuar = True
    for meneo in lista_meneos:
        dicc_aux = {}
        
        # Se obtiene en primer lugar el año
        fecha_pub = meneo.find("div", class_="news-submitted").find_all("span", class_="ts")[1].text.split(" ")[0]
        año = datetime.strptime(fecha_pub,"%d/%m/%Y").year if len(fecha_pub) > 5 else 2024
        
        # Si el año es el 2023, se continúa con la importación (para omitir las de 2024)
        if año == 2023:

            ###################################   Comunidad (Temática) ############################################################
            try:
                dicc_aux["Comunidad"] = meneo.find("a", class_="subname").text.strip()
            except:
                dicc_aux["Comunidad"] = np.nan

            ###################################    Usuario  ######################################################################
            try:
                dicc_aux["Usuario"] = meneo.find("div", class_="news-submitted").find("img")["alt"].strip()
            except:
                dicc_aux["Usuario"] = np.nan

            ###################################    Medio    ######################################################################
            try:
                dicc_aux["Medio"] = meneo.find("span", class_="showmytitle").text.strip()
            except:            
                dicc_aux["Medio"] = "meneame.net"

            ###################################    Fechas    ######################################################################    
            try:
                fechas = meneo.find("div", class_="news-submitted").find_all("span", class_="ts")
                dicc_aux["Enviado"] = fechas[0].text.strip()
                dicc_aux["Publicado"] = fechas[1].text.strip()
            except:
                dicc_aux["Enviado"] = np.nan
                dicc_aux["Publicado"] = np.nan

            ###################################    Titular    ####################################################################
            try:
                dicc_aux["Titular"] = meneo.find("h2").text.replace("\n", " ").strip()
            except:
                dicc_aux["Titular"] = np.nan

            ###################################    Entradilla    ###################################################################
            try:
                dicc_aux["Entradilla"] = meneo.find("div", class_="news-content").text.replace("\n", " ").strip()
            except:
                dicc_aux["Entradilla"] = np.nan
        
            ###################################    Votos    ######################################################################
            try:
                dicc_aux["Votos"] = meneo.find("div", class_="votes").find("a").text.strip()
            except:
                dicc_aux["Votos"] = np.nan

            ###################################    Click    ######################################################################
            try:
                dicc_aux["Clicks"] = meneo.find("div", class_="clics").find("span").text.strip()
            except:
                dicc_aux["Clicks"] = np.nan

            ###################################    Positivos    ###################################################################
            try:
                dicc_aux["Positivos"] = meneo.find("span", class_="positive-vote-number").text.strip()
            except:
                dicc_aux["Positivos"] = np.nan

            ###################################    Votos Anónimos    ##############################################################
            try:
                dicc_aux["Anonimos"] = meneo.find("span", class_="anonymous-vote-number").text.strip()
            except:
                dicc_aux["Anonimos"] = np.nan

            ###################################    Votos Negativos    ##############################################################
            try:
                dicc_aux["Negativos"] = meneo.find("span", class_="negative-vote-number").text.strip()
            except:
                dicc_aux["Negativos"] = np.nan

            ###################################    Votos Comentarios    #############################################################
            try:
                dicc_aux["Comentarios"] = meneo.find("a", class_="comments")["data-comments-number"].strip()
            except:
                dicc_aux["Comentarios"] = np.nan

            ###################################    Karma    ########################################################################
            try:
                dicc_aux["Karma"] = meneo.find("span", class_="karma-number").text.strip()
            except:
                dicc_aux["Karma"] = np.nan

            datos.append(dicc_aux)

        # Si se ha alcanzado el año es 2022, se detendrá la importación
        elif año == 2022:
            continuar = False

    return [datos, continuar]

In [5]:
######################################## BLOQUE PARA PRUEBAS ##############################################################

# Para obtener todo el año 2023, empezar con la pagina 128 y eliminar la condición de "contador" en el bucle ####

######################################## BLOQUE PARA PRUEBAS ##############################################################

browser = webdriver.Chrome()
pagina = 120
lista_aux = []
contador = 0
continuar = True

while contador < 10 and continuar == True:
    try:
        browser.get(f"https://old.meneame.net/?page={pagina}")        
        Soup_pagina_meneos = BeautifulSoup(browser.page_source, "html.parser")
        lista_meneos = Soup_pagina_meneos.find_all("div", class_="news-summary")
        resultado = extrae_datos_meneos(lista_meneos)
        lista_aux.extend(resultado[0])
        continuar = resultado[1]
        pagina += 1
        contador += 1
    except:
        print(f"Error en página {pagina}")    

print(f"Total paginas procesadas: {contador}")
print(f"Total meneos importados: {len(lista_aux)}")

browser.close()

Total paginas importadas: 615


In [ ]:
df = pd.DataFrame(lista_aux)
df

In [9]:

df.to_csv('meneame1.csv')


In [10]:

df.to_csv('meneame2.csv', index=False)


In [11]:

df.to_csv('meneame3.csv', sep=';', encoding='utf-8', index=False)
